In [2]:
import wordsfromweb as ww
import pandas as pd
import re
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize

In [3]:
pd.set_option('max_columns', 30)
pd.set_option('max_colwidth', 50)

In [4]:
ww.URLS['articles']

'http://bbc.com/mundo'

In [11]:
theurl = ww.URLS['articles']
s = ww.myget(theurl)
ll = ww.links_list(ww.article_links(s))
ldf = pd.DataFrame(ll, columns=['url','text'])
ldf['text'] = ldf['text'].str.strip()
ldf['furl'] = ldf['url'].apply(lambda x: x if x[0] =='h' else (theurl+x).replace('mundo/mundo','mundo')   )
link = ldf['furl'][0]

In [19]:
ldf.sample(10).sort_index()

,url,text,furl
7,/mundo/noticias-internacional-58268733,"""Mi hijo estaba tratando de ayudar a la región...",http://bbc.com/mundo/noticias-internacional-58...
12,/mundo/noticias-58325330,"""Yo no soy tu baterista, tú eres mi cantante"":...",http://bbc.com/mundo/noticias-58325330
15,/mundo/noticias-58329990,"5 de los mejores ritmos de Charlie Watts, el b...",http://bbc.com/mundo/noticias-58329990
18,/mundo/noticias-58322211,"5 preguntas sobre la variante delta, la más co...",http://bbc.com/mundo/noticias-58322211
26,/mundo/noticias-internacional-58308518,"""Estamos listos para la resistencia"": el grupo...",http://bbc.com/mundo/noticias-internacional-58...
28,/mundo/noticias-internacional-58243802,"El ""cementerio de imperios"": cómo los ejército...",http://bbc.com/mundo/noticias-internacional-58...
32,/mundo/noticias-america-latina-57923924,"El breve ""país"" que existió por menos de 3 año...",http://bbc.com/mundo/noticias-america-latina-5...
35,/mundo/noticias-america-latina-58206021,"Ernesto Samper: ""Nicaragua está comprometiendo...",http://bbc.com/mundo/noticias-america-latina-5...
37,/mundo/noticias-internacional-58290242,Ejecuciones sumarias y restricciones a mujeres...,http://bbc.com/mundo/noticias-internacional-58...
48,/mundo/noticias-internacional-58224969,"""Fue casi una estampida"": las imágenes que mue...",http://bbc.com/mundo/noticias-internacional-58...


In [20]:
ldf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   url     51 non-null     object
 1   text    51 non-null     object
 2   furl    51 non-null     object
dtypes: object(3)
memory usage: 1.3+ KB


In [110]:
article_dict = dict()
for i, row in ldf.iterrows():#replace with an applied function at some point?
    if i > 5: break #limit to a few urls for now
    s2 = None
    s2 = ww.myget(row['furl'])
    e = s2.find('main')
    #git rid of div tag noise
    for x in e.find_all('div'):
        x.replace_with_children()
    article_dict[row['text'] ] = e #row['furl'] #ultimately this should hold the 'main' part of the article
    
article_dict

{'Hay Festival': <main role="main"><h1 class="bbc-va1lce e1il82ea0" dir="ltr" id="content" tabindex="-1">Hay Festival Digital Querétaro @BBC Mundo</h1><section aria-labelledby="Lead-Feature" class="bbc-iinl4t e17m6msg2" role="region"><h2 class="bbc-1xclg2i e120qdcz0"><span class="bbc-1fttcpj e1vze7ki4"><span class="bbc-127x8fy e1vze7ki2"><span class="bbc-rtand8 e1vze7ki1" dir="ltr" id="Lead-Feature">Principales noticias</span></span></span></h2><ul class="e57qer20 bbc-10m7ymo eom0ln50" dir="ltr" role="list"><li class="e57qer20 bbc-1nfnx85 eom0ln51" dir="ltr"><h3 class="bbc-18u200t evnt13t1"><a class="bbc-1fxtbkn evnt13t0" href="/mundo/noticias-america-latina-51277086">"La sorpresa fue que el malestar en América Latina tardara tanto en manifestarse": entrevista con el nobel de Economía Joseph Stiglitz</a></h3><p class="bbc-1g5xny6 e1tfxkuo1">En entrevista con BBC Mundo, el economista estadounidense aconseja a los gobernantes latinoamericanos abordar rápido la desigualdad social y sugier

In [112]:
for k in article_dict.keys():
    print(k, len(article_dict[k]), len(article_dict[k].find_all('section')))

Hay Festival 10 9
Centroamérica Cuenta 7 6
Quiénes son las "Soñadoras afganas", el equipo adolescente de robótica que recibió asilo en México 52 2
A dónde se dirigen los refugiados de Afganistán y qué países los están acogiendo (también en América Latina) 98 2


### how can we save the articles so that we don't have to load them all the time while testing???

In [6]:
#move this into the module at some point (won't need the import)
import shelve
from bs4 import BeautifulSoup

def save_articles(article_dict):
    s = shelve.open('artdict')
    for k in article_dict.keys():
        s[k] = article_dict[k].prettify()
    s.close()

def load_saved_articles():

    d = dict()
    s=shelve.open('artdict')
    for k in s.keys():
        d[k] = BeautifulSoup(s[k]).find() #get it back to the same type as the original
    #
    s.close()
    return d

In [129]:
save_articles(article_dict)

In [7]:
d = load_saved_articles()

In [133]:
assert type(d[k]) == type(article_dict[k])

In [51]:
link = ldf.sample()['furl'].values[0]
print(f"getting {link}")
s2 = ww.myget(link)

getting http://bbc.com/mundo/noticias-58323375


In [52]:
#article text is always in the main tag
e = s2.find('main')
print(e.prettify())

<main role="main">
 <div class="bbc-1151pbn e57qer20" dir="ltr">
  <h1 class="bbc-1lsgtu3 e1yj3cbb0" id="content" tabindex="-1">
   Charlie Watts: muere el legendario baterista de los Rolling Stones a los 80 años
  </h1>
 </div>
 <div class="e1j2237y5 bbc-q4ibpr e57qer20" dir="ltr">
  <div class="bbc-1atl7vu e1c9i7u14">
   <ul class="bbc-cdrraw e1c9i7u12" role="list">
    <li class="bbc-1a3w4ok e1c9i7u11" role="listitem">
     Redacción
    </li>
    <li class="bbc-1p92jtu e1c9i7u10" role="listitem">
     BBC News Mundo
    </li>
   </ul>
  </div>
 </div>
 <div class="e1j2237y6 bbc-q4ibpr e57qer20" dir="ltr">
  <time class="bbc-14xtggo e4zesg50" datetime="2021-08-24">
   24 agosto 2021
  </time>
 </div>
 <div class="bbc-1ka88fa e57qer20" dir="ltr">
  <figure class="bbc-1qdcvv9 e6bmn90">
   <div class="bbc-172p16q e57qer20" dir="ltr">
    <div class="bbc-189y18v e57qer20" dir="ltr">
     <div class="bbc-1k5hvrj e1whu0" data-e2e="image-placeholder">
      <img alt="Charlie Watts" class="

In [70]:
#git rid of div tag noise
for x in e.find_all('div'):
    x.replace_with_children()

In [16]:
e = d[list(d.keys())[0]]

In [18]:
title = e.find('h1')
title.get_text().strip()

'Hay Festival Digital Querétaro @BBC Mundo'

In [19]:
#locate all the sections in main
for sec in e.find_all('section'):
    header = sec.find(re.compile('^h[1-6]$'))
    if header is None:
        print('no header found in section')
    else:
        print(header.get_text(strip=True))

Principales noticias
no header found in section
Destacamos
No te pierdas
Voces del Hay Festival
Cosas de la Lengua
Entrevistas
Literatura e Historia
Más del Hay Festival


In [73]:
print(e.get_text()[:300])

Charlie Watts: muere el legendario baterista de los Rolling Stones a los 80 añosRedacción  BBC News Mundo24 agosto 2021Fuente de la imagen, Getty ImagesEl baterista de los Rolling Stones, Charlie Watts, murió este martes a los 80 años, confirmó su publicista."Es con inmensa tristeza que anunciamos l


In [55]:
#this wasn't the issue - commenting out
#is image alt text being turned into text? e.g. ...agosto 2021Fuente de la imagen, Getty ImagesEl bater...
#for x in e.find_all('img'):
#    x.decompose()
#remove image tags and then check the text again

In [96]:
#print(e.getText()) #no duplicatse but some text is concatenated without spacing

In [50]:
lines = []
for x in e.find_all('p'):
    lines.append(x.get_text()) #this method could preserve space but will get duplicates

In [51]:
article_dict = dict()

In [52]:
#using set to de-dupe loses ordering - using a Series instead
cur_art_txt = " ".join(pd.Series(lines).drop_duplicates())#de dupe and put all text 
article_dict[title.get_text()]=cur_art_txt

In [55]:
from nltk.tokenize import WhitespaceTokenizer
print(WhitespaceTokenizer().tokenize(cur_art_txt))#retains quotes and punctuation

['En', 'entrevista', 'con', 'BBC', 'Mundo,', 'el', 'economista', 'estadounidense', 'aconseja', 'a', 'los', 'gobernantes', 'latinoamericanos', 'abordar', 'rápido', 'la', 'desigualdad', 'social', 'y', 'sugiere', 'una', 'reestructuración', 'de', 'la', 'deuda', 'de', 'Argentina', 'con', 'recorte', 'de', 'intereses.', 'La', 'novela', 'debut', 'de', 'Mateo', 'García', 'es', 'una', 'narración', 'diferente', 'sobre', 'la', 'problemática', 'de', 'las', 'drogas', 'en', 'México.', 'Se', 'trata', 'de', 'una', 'historia', 'más', 'personal,', 'de', 'cómo', 'un', 'personaje', 'se', 'va', 'despojando', 'de', 'todo', 'en', 'un', 'viaje', 'casi', 'voluntario', 'a', 'la', 'muerte', 'junto', 'a', 'la', 'heroína.', 'Para', 'la', 'francesa,', 'desparasitar', 'niños', 'o', 'mejorar', 'su', 'desarrollo', 'en', 'las', 'escuelas,', 'también', 'es', 'economía.', 'Duflo,', 'quien', 'ha', 'desarrollado', 'toda', 'su', 'carrera', 'en', 'el', 'MIT', 'y', 'fue', 'asesora', 'del', 'expresidente', 'Barack', 'Obama,,', 

In [56]:
from nltk.tokenize import RegexpTokenizer
tokens = RegexpTokenizer(r'\w+').tokenize(" ".join(pd.Series(lines).drop_duplicates()))
print(tokens)

['En', 'entrevista', 'con', 'BBC', 'Mundo', 'el', 'economista', 'estadounidense', 'aconseja', 'a', 'los', 'gobernantes', 'latinoamericanos', 'abordar', 'rápido', 'la', 'desigualdad', 'social', 'y', 'sugiere', 'una', 'reestructuración', 'de', 'la', 'deuda', 'de', 'Argentina', 'con', 'recorte', 'de', 'intereses', 'La', 'novela', 'debut', 'de', 'Mateo', 'García', 'es', 'una', 'narración', 'diferente', 'sobre', 'la', 'problemática', 'de', 'las', 'drogas', 'en', 'México', 'Se', 'trata', 'de', 'una', 'historia', 'más', 'personal', 'de', 'cómo', 'un', 'personaje', 'se', 'va', 'despojando', 'de', 'todo', 'en', 'un', 'viaje', 'casi', 'voluntario', 'a', 'la', 'muerte', 'junto', 'a', 'la', 'heroína', 'Para', 'la', 'francesa', 'desparasitar', 'niños', 'o', 'mejorar', 'su', 'desarrollo', 'en', 'las', 'escuelas', 'también', 'es', 'economía', 'Duflo', 'quien', 'ha', 'desarrollado', 'toda', 'su', 'carrera', 'en', 'el', 'MIT', 'y', 'fue', 'asesora', 'del', 'expresidente', 'Barack', 'Obama', 'es', 'una'

## dealing with proper nouns

In [175]:
#proper nouns - only english and russian are supported by pos_tag
#from nltk.tag import pos_tag
#pos_tag(tokens, lang='esp')

In [57]:
import es_core_news_sm
nlp = es_core_news_sm.load()
txt = """Charlie Watts: muere el legendario baterista de los Rolling Stones a los 80 años Redacción  
BBC News Mundo 24 agosto 2021 Fuente de la imagen, Getty Images El baterista de los Rolling Stones, Charlie Watts,
murió este martes a los 80 años, confirmó su publicista. "Es con inmensa tristeza que anunciamos"
"""
doc = nlp(cur_art_txt)
print([(w.text, w.pos_) for w in doc])

[('\n     ', 'SPACE'), ('En', 'ADP'), ('entrevista', 'NOUN'), ('con', 'ADP'), ('BBC', 'PROPN'), ('Mundo', 'PROPN'), (',', 'PUNCT'), ('el', 'DET'), ('economista', 'NOUN'), ('estadounidense', 'ADJ'), ('aconseja', 'VERB'), ('a', 'ADP'), ('los', 'DET'), ('gobernantes', 'NOUN'), ('latinoamericanos', 'ADJ'), ('abordar', 'VERB'), ('rápido', 'ADV'), ('la', 'DET'), ('desigualdad', 'NOUN'), ('social', 'ADJ'), ('y', 'CCONJ'), ('sugiere', 'VERB'), ('una', 'DET'), ('reestructuración', 'NOUN'), ('de', 'ADP'), ('la', 'DET'), ('deuda', 'NOUN'), ('de', 'ADP'), ('Argentina', 'PROPN'), ('con', 'ADP'), ('recorte', 'NOUN'), ('de', 'ADP'), ('intereses', 'NOUN'), ('.', 'PUNCT'), ('\n     \n     ', 'SPACE'), ('La', 'DET'), ('novela', 'NOUN'), ('debut', 'NOUN'), ('de', 'ADP'), ('Mateo', 'PROPN'), ('García', 'PROPN'), ('es', 'AUX'), ('una', 'DET'), ('narración', 'NOUN'), ('diferente', 'ADJ'), ('sobre', 'ADP'), ('la', 'DET'), ('problemática', 'NOUN'), ('de', 'ADP'), ('las', 'DET'), ('drogas', 'NOUN'), ('en', 'AD

In [84]:
sw = nlp.Defaults.stop_words
print(len(sw), sw)

551 {'dijo', 'que', 'verdad', 'después', 'más', 'pronto', 'ademas', 'entonces', 'consiguen', 'partir', 'deprisa', 'éstas', 'vosotros', 'ver', 'algo', 'fue', 'consideró', 'hago', 'ningunas', 'usar', 'ya', 'demás', 'poner', 'deben', 'vamos', 'nosotras', 'general', 'lado', 'agregó', 'cuántas', 'ustedes', 'mí', 'porque', 'les', 'buenos', 'podrían', 'dejó', 'vuestro', 'todo', 'sino', 'junto', 'buena', 'tus', 'pocos', 'uso', 'cierto', 'sobre', 'tú', 'poca', 'aquellos', 'acuerdo', 'según', 'sus', 'quiza', 'podemos', 'estados', 'vais', 'saber', 'habla', 'poder', 'han', 'hacemos', 'cuanto', 'estar', 'pueden', 'mía', 'ser', 'nuestra', 'nunca', 'alguna', 'haciendo', 'quizá', 'cuanta', 'cerca', 'cómo', 'quizás', 'fuera', 'está', 'unos', 'soyos', 'claro', 'cierta', 'ti', 'no', 'cual', 'último', 'fui', 'haces', 'mias', 'fin', 'estan', 'igual', 'tenga', 'enseguida', 'cuáles', 'también', 'somos', 'debido', 'arribaabajo', 'cuales', 'parte', 'toda', 'nueva', 'verdadera', 'tuvo', 'aquél', 'diferente', 'h

In [64]:
print([w for w in doc if w.pos_ not in ['SPACE', 'PUNCT', 'PROPN']])

[En, entrevista, con, el, economista, estadounidense, aconseja, a, los, gobernantes, latinoamericanos, abordar, rápido, la, desigualdad, social, y, sugiere, una, reestructuración, de, la, deuda, de, con, recorte, de, intereses, La, novela, debut, de, es, una, narración, diferente, sobre, la, problemática, de, las, drogas, en, Se, trata, de, una, historia, más, personal, de, cómo, un, personaje, se, va, despojando, de, todo, en, un, viaje, casi, voluntario, a, la, muerte, junto, a, la, heroína, Para, la, francesa, desparasitar, niños, o, mejorar, su, desarrollo, en, las, escuelas, también, es, economía, quien, ha, desarrollado, toda, su, carrera, en, el, y, fue, asesora, del, expresidente, es, una, economista, atípica, que, ha, sacado, la, teoría, a, la, calle, Éxito, de, ventas, y, también, de, crítica, es, sin, duda, uno, de, los, principales, representantes, de, la, literatura, francesa, contemporánea, conversó, con, él, con, ocasión, del, de, Al, premio, de, de, 2008, le, cambió, la

In [39]:
w = doc[0]
print([x for x in dir(w) if x[0] != '_'])

['ancestors', 'check_flag', 'children', 'cluster', 'conjuncts', 'dep', 'dep_', 'doc', 'ent_id', 'ent_id_', 'ent_iob', 'ent_iob_', 'ent_kb_id', 'ent_kb_id_', 'ent_type', 'ent_type_', 'get_extension', 'has_dep', 'has_extension', 'has_head', 'has_morph', 'has_vector', 'head', 'i', 'idx', 'iob_strings', 'is_alpha', 'is_ancestor', 'is_ascii', 'is_bracket', 'is_currency', 'is_digit', 'is_left_punct', 'is_lower', 'is_oov', 'is_punct', 'is_quote', 'is_right_punct', 'is_sent_end', 'is_sent_start', 'is_space', 'is_stop', 'is_title', 'is_upper', 'lang', 'lang_', 'left_edge', 'lefts', 'lemma', 'lemma_', 'lex', 'lex_id', 'like_email', 'like_num', 'like_url', 'lower', 'lower_', 'morph', 'n_lefts', 'n_rights', 'nbor', 'norm', 'norm_', 'orth', 'orth_', 'pos', 'pos_', 'prefix', 'prefix_', 'prob', 'rank', 'remove_extension', 'right_edge', 'rights', 'sent', 'sent_start', 'sentiment', 'set_extension', 'set_morph', 'shape', 'shape_', 'similarity', 'subtree', 'suffix', 'suffix_', 'tag', 'tag_', 'tensor', 't

In [60]:
doc.sentiment

0.0

In [110]:
# 'to' and 'and' are very common - needed for context but not for frequency
# tf/idf classification would inherently remove stopwords because they would be frequent in every document
sw = sw.union({'a', 'y'})

In [111]:
posdf = pd.DataFrame([(w.text, w.pos_, w.lemma_, w.lemma_ in sw) for w in doc], columns=['word','POS','lemma', 'stopword'])
posdf[posdf['POS'] == 'PROPN']

,word,POS,lemma,stopword
4,BBC,PROPN,BBC,False
5,Mundo,PROPN,Mundo,False
28,Argentina,PROPN,Argentina,False
39,Mateo,PROPN,Mateo,False
40,García,PROPN,García,False
...,...,...,...,...
1403,Festival,PROPN,Festival,False
1427,BBC,PROPN,BBC,False
1428,Mundo,PROPN,Mundo,False
1435,Arequipa,PROPN,Arequipa,False


In [112]:
posdf['stopword'].value_counts()

False    770
True     670
Name: stopword, dtype: int64

In [113]:
posdf2 = posdf[posdf['stopword']==False]
posdf2.sample(10)

,word,POS,lemma,stopword
382,\n \n,SPACE,\n \n,False
826,.,PUNCT,.,False
1436,",",PUNCT,",",False
26,deuda,NOUN,deuda,False
483,argentino,NOUN,argentino,False
956,necesario,ADJ,necesario,False
65,personaje,NOUN,personaje,False
1245,millones,NOUN,millón,False
374,enemigos,NOUN,enemigo,False
144,",",PUNCT,",",False


In [114]:
poswords = set(posdf['POS']) - {'PUNCT', 'PROPN','SPACE'}

In [115]:
posdf2 = posdf2[posdf2['POS'].isin(poswords)]
posdf2.sample(10)

,word,POS,lemma,stopword
971,letras,NOUN,letra,False
1119,imaginación,NOUN,imaginación,False
1142,significa,VERB,significar,False
1112,mexicano,ADJ,mexicano,False
1181,público,NOUN,público,False
637,habla,VERB,hablar,False
331,sociedad,NOUN,sociedad,False
643,pandemias,NOUN,pandemia,False
365,gente,NOUN,gente,False
218,peruano,ADJ,peruano,False


In [117]:
#most common stemmed words
posdf2['lemma'].value_counts()[:10]

hablar        8
vida          7
novela        5
mexicano      5
historia      5
escritor      5
entrevista    4
obra          4
lengua        4
mundo         3
Name: lemma, dtype: int64

### stemming an lemmatization
https://medium.com/mlearning-ai/nlp-03-lemmatization-and-stemming-using-spacy-b2829becceca

spaCy does not contain any function for stemming. spaCy provides lemmatization via dictionary lookup and each language has its own dictionary. 


In [170]:
#remove stopwords
from nltk.corpus import stopwords
stopwrds = stopwords.words('spanish')
tokens2 = [x for x in tokens if x not in stopwrds]

In [171]:
s = SnowballStemmer('spanish', ignore_stopwords=True)

st = list(map(s.stem, tokens2)) #stems names too
st

['fuent',
 'imag',
 'getty',
 'imag',
 'el',
 'bater',
 'rolling',
 'ston',
 'charli',
 'watts',
 'mur',
 'mart',
 '80',
 'años',
 'confirm',
 'public',
 'es',
 'inmens',
 'tristez',
 'anunci',
 'muert',
 'amad',
 'charli',
 'watts',
 'lee',
 'comunic',
 'divulg',
 'medi',
 'represent',
 'fallec',
 'hoy',
 'pacif',
 'hospital',
 'londr',
 'rod',
 'famili',
 'añad',
 'el',
 'comunic',
 'dic',
 'watts',
 'quer',
 'espos',
 'padr',
 'abuel',
 'grand',
 'bater',
 'gener',
 'pid',
 'respet',
 'intim',
 'famili',
 'amig',
 'moment',
 'dificil',
 'la',
 'band',
 'previst',
 'reanud',
 'gir',
 'ee',
 'uu',
 'septiembr',
 'despues',
 'vers',
 'oblig',
 'posterg',
 'año',
 'pas',
 'pandemi',
 'coronavirus',
 'final',
 'quizas',
 'interes',
 'a',
 'principi',
 'agost',
 'grup',
 'comunic',
 'stev',
 'jord',
 'sustitut',
 'watts',
 'gir',
 'estadounidens',
 'fuent',
 'imag',
 'pa',
 'medi',
 'los',
 'rolling',
 'ston',
 '1969',
 'charli',
 'watts',
 'miembr',
 'ston',
 'ener',
 '1963',
 'uni',
 'm

In [172]:
pd.Series(st).value_counts()[:10]

watts      21
ston       13
charli     11
band        8
años        7
rolling     6
bater       5
dij         5
the         5
toc         4
dtype: int64

In [143]:
#tokens = word_tokenize(" ".join(pd.Series(lines).drop_duplicates()))#includes punctuation

['Fuente',
 'de',
 'la',
 'imagen',
 ',',
 'Getty',
 'Images',
 'El',
 'baterista',
 'de',
 'los',
 'Rolling',
 'Stones',
 ',',
 'Charlie',
 'Watts',
 ',',
 'murió',
 'este',
 'martes',
 'a',
 'los',
 '80',
 'años',
 ',',
 'confirmó',
 'su',
 'publicista',
 '.',
 '``',
 'Es',
 'con',
 'inmensa',
 'tristeza',
 'que',
 'anunciamos',
 'la',
 'muerte',
 'de',
 'nuestro',
 'amado',
 'Charlie',
 'Watts',
 "''",
 ',',
 'se',
 'lee',
 'en',
 'el',
 'comunicado',
 'divulgado',
 'a',
 'los',
 'medios',
 'por',
 'su',
 'representante',
 '.',
 '``',
 'Falleció',
 'hoy',
 'pacíficamente',
 'en',
 'un',
 'hospital',
 'de',
 'Londres',
 ',',
 'rodeado',
 'de',
 'su',
 'familia',
 "''",
 ',',
 'añade',
 '.',
 'El',
 'comunicado',
 'también',
 'dice',
 'que',
 'Watts',
 'fue',
 '``',
 'un',
 'querido',
 'esposo',
 ',',
 'padre',
 'y',
 'abuelo',
 "''",
 'y',
 '``',
 'uno',
 'de',
 'los',
 'más',
 'grandes',
 'bateristas',
 'de',
 'su',
 'generación',
 "''",
 'y',
 'pide',
 'que',
 'se',
 'respete',
 'l

In [100]:
print(e.prettify())

<main role="main">
 <h1 class="bbc-1lsgtu3 e1yj3cbb0" id="content" tabindex="-1">
  Charlie Watts: muere el legendario baterista de los Rolling Stones a los 80 años
 </h1>
 <ul class="bbc-cdrraw e1c9i7u12" role="list">
  <li class="bbc-1a3w4ok e1c9i7u11" role="listitem">
   Redacción
  </li>
  <li class="bbc-1p92jtu e1c9i7u10" role="listitem">
   BBC News Mundo
  </li>
 </ul>
 <time class="bbc-14xtggo e4zesg50" datetime="2021-08-24">
  24 agosto 2021
 </time>
 <figure class="bbc-1qdcvv9 e6bmn90">
  <p class="bbc-1s1cxbv etq3yw90" role="text">
   <span class="bbc-1gnhmg2 e1yt7oin0">
    Fuente de la imagen,
   </span>
   <span lang="en-GB">
    Getty Images
   </span>
  </p>
 </figure>
 <p class="bbc-bm53ic e1cc2ql70" dir="ltr">
  <b>
   El baterista de los Rolling Stones, Charlie Watts, murió este martes a los 80 años, confirmó su publicista.
  </b>
 </p>
 <p class="bbc-bm53ic e1cc2ql70" dir="ltr">
  "Es con inmensa tristeza que
  <b>
   anunciamos la muerte de nuestro amado Charlie Wa